<a href="https://colab.research.google.com/github/sujitkoley9/Sentiment-Analysis/blob/master/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1> <center>Sentiment Analysis </center> </h1>
<b>Useful link: </b>


1. https://www.analyticsvidhya.com/blog/2018/04/a-comprehensive-guide-to-understand-and-implement-text-classification-in-python/
2. https://realpython.com/python-keras-text-classification/

3. https://medium.com/jatana/report-on-text-classification-using-cnn-rnn-han-f0e887214d5f

4. https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/


# 1. **Importing packages**



In [0]:
from datetime import date, timedelta,datetime
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')

import numpy as np
from nltk.corpus import stopwords,wordnet
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize,pos_tag

import gensim
from gensim.test.utils import get_tmpfile
from gensim.summarization import keywords

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.model_selection import RandomizedSearchCV


from tensorflow.keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from tensorflow.keras import layers
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout,Flatten
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Embedding, LSTM, GRU
from tensorflow.keras.layers import Embedding
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

# 2 . **Reading required files**

In [0]:
movie_df = pd.read_csv('amazon_cells_labelled.txt', names=['review', 'sentiment'],sep='\t')

# 1. **Logistic Regression**


In [7]:
# Cleaning the text
X = movie_df['review']
y = movie_df['sentiment']



# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1000)

#  fit the vectorizer
vectorizer = TfidfVectorizer(max_df =1.0,min_df=1,analyzer='word')
fit_obj = vectorizer.fit(X_train)

# Transform the vectorizer
X_train = fit_obj.transform(X_train)
X_test = fit_obj.transform(X_test)






classifier = LogisticRegression()
classifier.fit(X_train, y_train)
score = classifier.score(X_test, y_test)
print("Accuracy:", score)

Accuracy: 0.8


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


# 2. **Neural network without word embedding**


In [8]:
# Cleaning the text
X = movie_df['review']
y = movie_df['sentiment']



# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1000)

#  fit the vectorizer
vectorizer = TfidfVectorizer(max_df =1.0,min_df=1,analyzer='word')
fit_obj = vectorizer.fit(X_train)

# Transform the vectorizer
X_train = fit_obj.transform(X_train)
X_test = fit_obj.transform(X_test)

# Neural Network 
classifier  = Sequential()
classifier.add(Dense(10,input_shape=(X_train.shape[1],), activation = 'relu'))
classifier.add(Dense(1, activation = 'sigmoid'))

# model summary
classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10)                15470     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 11        
Total params: 15,481
Trainable params: 15,481
Non-trainable params: 0
_________________________________________________________________


In [9]:
# compile the model
classifier.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])


# Train the model
history = classifier.fit(X_train, y_train,
                    epochs=100,
                    verbose=1 ,
                    validation_data=(X_test, y_test),
                    batch_size=X_train.shape[0])

Train on 750 samples, validate on 250 samples
Epoch 1/100
750/750 [==============================] - 0s 151us/sample - loss: 0.6926 - accuracy: 0.5120 - val_loss: 0.6917 - val_accuracy: 0.5600
Epoch 2/100
750/750 [==============================] - 0s 19us/sample - loss: 0.6910 - accuracy: 0.5533 - val_loss: 0.6911 - val_accuracy: 0.5640
Epoch 3/100
750/750 [==============================] - 0s 18us/sample - loss: 0.6894 - accuracy: 0.5893 - val_loss: 0.6905 - val_accuracy: 0.5760
Epoch 4/100
750/750 [==============================] - 0s 16us/sample - loss: 0.6877 - accuracy: 0.6293 - val_loss: 0.6898 - val_accuracy: 0.5960
Epoch 5/100
750/750 [==============================] - 0s 17us/sample - loss: 0.6861 - accuracy: 0.6600 - val_loss: 0.6891 - val_accuracy: 0.5960
Epoch 6/100
750/750 [==============================] - 0s 15us/sample - loss: 0.6845 - accuracy: 0.6973 - val_loss: 0.6885 - val_accuracy: 0.6120
Epoch 7/100
750/750 [==============================] - 0s 19us/sample - loss:

In [10]:
loss, accuracy = classifier.evaluate(X_test, y_test, verbose=False)
print("Accuracy:", accuracy)

Accuracy: 0.736


# 3. **Neural network with word embedding**


## 3.1 **LSTM**


In [0]:
# Cleaning the text
X = movie_df['review']
y = movie_df['sentiment']

# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1000)

# fit method
tokenizer_obj = Tokenizer()
tokenizer_obj.fit_on_texts(X_train) 

# pad sequences
max_length = max([len(s.split()) for s in X_train])

# define vocabulary size
vocab_size = len(tokenizer_obj.word_index) + 1


# Transform
X_train_tokens =  tokenizer_obj.texts_to_sequences(X_train)
X_test_tokens = tokenizer_obj.texts_to_sequences(X_test)

# Padding
X_train_pad = pad_sequences(X_train_tokens, maxlen=max_length, padding='post')
X_test_pad = pad_sequences(X_test_tokens, maxlen=max_length, padding='post')

In [36]:
# LSTM
embedding_dim =100
classifier  = Sequential()
classifier.add(Embedding(input_dim=vocab_size, 
                         output_dim=embedding_dim, 
                         input_length=max_length))

classifier.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))

classifier.add(Dense(10, activation = 'relu'))

classifier.add(Dense(1, activation = 'sigmoid'))

# model summary
classifier.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 30, 100)           157400    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               80400     
_________________________________________________________________
dense_10 (Dense)             (None, 10)                1010      
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 11        
Total params: 238,821
Trainable params: 238,821
Non-trainable params: 0
_________________________________________________________________


In [37]:
# compile the model
classifier.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
# Train the model
history = classifier.fit(X_train_pad, np.array(y_train),
                    epochs=100,
                    verbose=1 ,
                    validation_data=(X_test_pad, np.array(y_test)),
                    batch_size=X_test_pad.shape[0])

Train on 750 samples, validate on 250 samples
Epoch 1/100
750/750 [==============================] - 2s 3ms/sample - loss: 0.6929 - accuracy: 0.5107 - val_loss: 0.6931 - val_accuracy: 0.5040
Epoch 2/100
750/750 [==============================] - 0s 614us/sample - loss: 0.6934 - accuracy: 0.4933 - val_loss: 0.6931 - val_accuracy: 0.5040
Epoch 3/100
750/750 [==============================] - 0s 654us/sample - loss: 0.6937 - accuracy: 0.4827 - val_loss: 0.6930 - val_accuracy: 0.5040
Epoch 4/100
750/750 [==============================] - 0s 626us/sample - loss: 0.6928 - accuracy: 0.5133 - val_loss: 0.6932 - val_accuracy: 0.4960
Epoch 5/100
750/750 [==============================] - 0s 649us/sample - loss: 0.6923 - accuracy: 0.5347 - val_loss: 0.6931 - val_accuracy: 0.4960
Epoch 6/100
750/750 [==============================] - 0s 636us/sample - loss: 0.6920 - accuracy: 0.5507 - val_loss: 0.6929 - val_accuracy: 0.5120
Epoch 7/100
750/750 [==============================] - 0s 632us/sample - l

In [38]:
loss, accuracy = classifier.evaluate(X_test_pad, np.array(y_test), verbose=True)
print("Accuracy:", accuracy)

250/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

## 3.2 **GRU**

In [0]:
# Cleaning the text
X = movie_df['review']
y = movie_df['sentiment']

# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1000)

# fit method
tokenizer_obj = Tokenizer()
tokenizer_obj.fit_on_texts(X_train) 

# pad sequences
max_length = max([len(s.split()) for s in X_train])

# define vocabulary size
vocab_size = len(tokenizer_obj.word_index) + 1


# Transform
X_train_tokens =  tokenizer_obj.texts_to_sequences(X_train)
X_test_tokens = tokenizer_obj.texts_to_sequences(X_test)

# Padding
X_train_pad = pad_sequences(X_train_tokens, maxlen=max_length, padding='post')
X_test_pad = pad_sequences(X_test_tokens, maxlen=max_length, padding='post')

In [40]:
# LSTM
embedding_dim =100
classifier  = Sequential()
classifier.add(Embedding(input_dim=vocab_size, 
                         output_dim=embedding_dim, 
                         input_length=max_length))

classifier.add(GRU(100, dropout=0.2, recurrent_dropout=0.2))

classifier.add(Dense(10, activation = 'relu'))

classifier.add(Dense(1, activation = 'sigmoid'))

# model summary
classifier.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 30, 100)           157400    
_________________________________________________________________
gru (GRU)                    (None, 100)               60600     
_________________________________________________________________
dense_12 (Dense)             (None, 10)                1010      
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 11        
Total params: 219,021
Trainable params: 219,021
Non-trainable params: 0
_________________________________________________________________


In [41]:
# compile the model
classifier.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
# Train the model
history = classifier.fit(X_train_pad, np.array(y_train),
                    epochs=100,
                    verbose=1 ,
                    validation_data=(X_test_pad, np.array(y_test)),
                    batch_size=X_test_pad.shape[0])

Train on 750 samples, validate on 250 samples
Epoch 1/100
750/750 [==============================] - 2s 3ms/sample - loss: 0.6938 - accuracy: 0.4880 - val_loss: 0.6931 - val_accuracy: 0.5040
Epoch 2/100
750/750 [==============================] - 1s 743us/sample - loss: 0.6935 - accuracy: 0.4867 - val_loss: 0.6933 - val_accuracy: 0.4960
Epoch 3/100
750/750 [==============================] - 1s 760us/sample - loss: 0.6933 - accuracy: 0.4787 - val_loss: 0.6934 - val_accuracy: 0.4960
Epoch 4/100
750/750 [==============================] - 1s 774us/sample - loss: 0.6927 - accuracy: 0.5040 - val_loss: 0.6932 - val_accuracy: 0.4960
Epoch 5/100
750/750 [==============================] - 1s 720us/sample - loss: 0.6935 - accuracy: 0.4640 - val_loss: 0.6932 - val_accuracy: 0.4920
Epoch 6/100
750/750 [==============================] - 1s 736us/sample - loss: 0.6923 - accuracy: 0.5307 - val_loss: 0.6931 - val_accuracy: 0.5040
Epoch 7/100
750/750 [==============================] - 1s 729us/sample - l

In [42]:
loss, accuracy = classifier.evaluate(X_test_pad, np.array(y_test), verbose=True)
print("Accuracy:", accuracy)

250/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================